In [1]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types_lab9 import CMD
import time
import numpy as np
LOG.propagate = False

In [23]:
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2025-04-14 19:25:06,771 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:81:04:26:a3:64


2025-04-14 19:25:09,947 | INFO     |: Connected to c0:81:04:26:a3:64


In [ ]:
ble.disconnect()

In [24]:
ble.send_command(CMD.PING, "")
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

PONG


In [16]:
ble.send_command(CMD.SET_PID_GAINS, "2.5|0.02|0.7")

In [25]:
ble.send_command(CMD.START_PID, "")

In [26]:
ble.send_command(CMD.STOP_PID, "")

In [31]:
results = []
# ble.stop_notify(ble.uuid['RX_STRING'])
def notification_handler(uuid, byte_array):
    time, imu, pwm, tof= ble.bytearray_to_string(byte_array).split('|')
    results.append([float(time), float(imu), int(pwm), float(tof)])


ble.start_notify(ble.uuid['RX_STRING'], notification_handler)
ble.send_command(CMD.SEND_PID_DATA, "")

In [30]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [32]:
print(results)

[]


2025-04-14 19:26:27,345 | INFO     |: Disconnected from A1C0DD95-B611-6CFA-AF80-6BCF50487F59


In [ ]:
import csv
import datetime

# Generate filename with timestamp to avoid overwriting
angle = '(-2,1)'
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"robot_data_{timestamp}_{angle}.csv"

with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write header
    csvwriter.writerow(['Time (ms)', 'IMU', 'PWM', 'TOF'])
    # Write all data rows
    csvwriter.writerows(results)

print(f"Data saved to {filename}")

In [ ]:
imu_angles_deg = [row[1] for row in results]
tof_readings = [row[3] for row in results]

# Convert angles from degrees to radians for plotting
imu_angles_rad = np.radians(imu_angles_deg)

# Create the polar plot
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
# 'o-' plots points and connects them
ax.plot(imu_angles_rad, tof_readings, 'o-')

# Optional: Set plot limits or labels
# Adjust radial limit slightly beyond max reading
ax.set_rmax(max(tof_readings) * 1.1)
ax.set_rticks(np.linspace(0, ax.get_rmax(), 5))  # Example radial ticks
ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
# Set angular limits if needed
ax.set_thetalim(min(imu_angles_rad), max(imu_angles_rad))
ax.grid(True)

ax.set_title("TOF Distance vs. IMU Angle", va='bottom')
plt.show()